### Implementing the Failed Logins Analytic in Production

For this notebook, we demonstrate how the queries and data we saw in the previous exploratory notebook can be used to implement the same analytic but in a production alerting environment.

Note that here we import a number of prebuilt modules from the `sascyber` module.

Additionally, we use a custom analytic that we create and store in the `/home/ds/custom/code/` path.

__Damian Herrick__  
__SAS Institute__  
__[damian.herrick@sas.com](mailto:damian.herrick@sas.com)__  

In [1]:
import os
import sys

from sascyber.utils.log import setup_logging
from sascyber.configs.analyticmanager import AnalyticsManagerConfig
from sascyber.servers.DAGServer import DAGServer
from sascyber.utils.exceptions import SASCyberMissingEnvVariable
from sascyber.utils.exceptions import SASCyberMissingConfig

Our base analytic for this is called `Logins`

In [2]:
from sascyber.analytics.wh.logins import Logins

The production analytic environment requires a lot more in terms of environment variables:

* `CYBER_ARGS` defines the analytic configuration file.
* `CYBER_ROOT` defines the root directory.
* `CYBER_DEFAULT_LOGGING` defines whether we use logging defaults. We don't here.
* `CYBER_LOG` defines the location of the logging format configuration file.

In [3]:
os.environ["CYBER_ARGS"] = '/home/ds/custom/config/wh_60m.json'
os.environ["CYBER_ROOT"] = '/home/ds'
os.environ["CYBER_DEFAULT_LOGGING"] = 'false'
os.environ["CYBER_LOG"] = '/home/ds/custom/config/logformat.json'

First thing we need to do is create a configuration class.

This class orchestrates the analytics as they run - consolidates everything we need into a single object.

In [4]:
cyber_config = AnalyticsManagerConfig()

If it successfully created, we found the configuration.

Because we're doing this interactively, report the status of the file. Typically this isn't done in production, though.

In [5]:
cyber_config.status()

CYBER_ARGS (environment variable) :: /home/ds/custom/config/wh_60m.json
CYBER_ARGS (config object)        :: /home/ds/custom/config/wh_60m.json
CYBER_ROOT (environment variable) :: /home/ds
CYBER_ROOT (config object)        :: /home/ds
CYBER_DEFAULT_LOGGING (environment variable) :: false
CYBER_DEFAULT_LOGGING (config object)        :: False
CYBER_LOG (environment variable) :: /home/ds/custom/config/logformat.json
CYBER_LOG (config object)        :: /home/ds/custom/config/logformat.json
analytics_type: sascyber.analytics
custom_analytics_allowed: True
custom_analytics_path: /home/ds/custom/wh
filters: {
    "EventsFailedLogins": {
        "clause": "EventID = 4625",
        "input": "{intable}",
        "output": "{intable}_FAILED_LOGINS"
    }
}
loginfo: {
    "formatters": {
        "f": {
            "datefmt": "%Y-%m-%d::%H:%M:%S",
            "format": "%(asctime)s:\t%(process)d:\t%(filename)s:\t%(levelname)s:\t[%(name)s:%(filename)s:%(lineno)s - %(funcName)s()] %(message)s"
     

Next, we change to our cyber root directory.

In [6]:
os.chdir(cyber_config.CYBER_ROOT)

Set up our logging.

In [7]:
log = setup_logging(cyber_config)
log.info("initialization successful")

Our analytics are run using a `DAGServer` object. This object handles all of the heavy lifting of our analytic.

In [ ]:
server = DAGServer(cyber_config)

Finally, we run our analytic. This loads data from CAS, calculates the analytic scores for each entity (USERS in this case), and then writes all to a CSV.

In [ ]:
server.run()